In [1]:
import pandas as pd
import numpy as np
import math
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
from decimal import Decimal
import datetime
import os
pd.set_option('display.max_columns', None)
#DB Details
connection_type = "mssql+pyodbc"
dataportal_db_user = "dataportal"
dataportal_db_password = "FatShamingMarc"
dataportal_db_ip = "10.8.4.35"
db_port = "1433"
db_prod_name = "dataportal_prod"
db_driver = "ODBC+DRIVER+17+for+SQL+Server"

In [2]:
def create_connection_string(connection_type, user, password, host, port, database_name, driver):
    return (
        "{connection_type}://{user}:{password}@{host}:{port}/{database_name}"
        "?driver={driver}").format(
        connection_type=connection_type,
        user=user,
        password=password,
        host=host,
        port=port,
        database_name=database_name,
        driver=driver)

##### Load for Offline Events file

In [3]:
filename = 'OfflineEvents20200825.csv'
path = 'C:\\Temp\\IIR\\'
filepath = path + filename

In [8]:
def mapOfflineEventsDataframe(df, filename):
    df.loc[df.PADD_REG == 'I', 'PADD_REG'] = "PAD District 1"
    df.loc[df.PADD_REG == 'II', 'PADD_REG'] = "PAD District 2"
    df.loc[df.PADD_REG == 'III', 'PADD_REG'] = "PAD District 3"
    df.loc[df.PADD_REG == 'IV', 'PADD_REG'] = "PAD District 4"
    df.loc[df.PADD_REG == 'V', 'PADD_REG'] = "PAD District 5"
    df['ORIGIN_FILE'] = filename

In [16]:
#df['COMMENTS'] = df['COMMENTS'].fillna('').str.replace("'", "''")
#df['AREA_NAME'] = df['AREA_NAME'].str.replace("'", "''")
#df['OWNER_NAME'] = df['OWNER_NAME'].str.replace("'", "''")
#df['PLANT_NAME'] = df['OWNER_NAME'].str.replace("'", "''")
# def insertOfflineEvents_CsvToSql_Fast(engine, path, filename, columns):
#     filepath = path + filename
#     df = pd.read_csv(filepath, low_memory=False)[columns]
#     mapOfflineEventsDataframe(df, filename)
#     queryText = ''
#     for index, row in df.iterrows():
#         queryText += \
# f"""INSERT INTO dataportal_prod..data_iir_offlineevents values ('{row.RELEASE_DT}',{int(row.EVENT_ID)},'{row.UNIT_NAME}',{int(row.UNIT_ID)},{int(row.AREA_ID)},'{row.AREA_NAME}','{row.OWNER_NAME}',\
# {int(row.PLANT_ID)},'{row.PLANT_NAME}','{row.U_STATUS}','{row.UTYPE_DESC}',{"NULL" if pd.isna(row.U_CAPACITY) else int(row.U_CAPACITY)},\
# {"NULL" if pd.isna(row.CAP_OFFLINE) else int(row.CAP_OFFLINE)},'{row.START_DATE}','{row.END_DATE}','{row.PRECISION}','{row.EVENT_TYPE}',\
# '{row.E_STATUS}','{"NULL" if pd.isna(row.E_CAUSE) else row.E_CAUSE}',{int(row.E_DURATION)},'{row.PADD_REG}','{"NULL" if pd.isna(row.COMMENTS) else row.COMMENTS}','{row.ORIGIN_FILE}');\n"""

#         if index % 250 == 0 or index == df.shape[0] - 1:
#             #print(index)
#             with engine.begin() as connection:
#                 connection.execute(queryText)
#                 queryText=''

In [4]:
#Define columns we're interested in from the file
columns=['RELEASE_DT', 'EVENT_ID', 'UNIT_NAME', 'UNIT_ID', 'AREA_ID', 'AREA_NAME', 'OWNER_NAME', 'PLANT_ID', 'PLANT_NAME', 
         'U_STATUS', 'UTYPE_DESC', 'U_CAPACITY', 'CAP_OFFLINE', 'START_DATE', 'END_DATE', 'PRECISION', 'EVENT_TYPE', 
         'E_STATUS', 'E_CAUSE', 'E_DURATION', 'PADD_REG', 'COMMENTS']

In [5]:
renamed_columns={"RELEASE_DT": "PUBLISH_DATE", "U_STATUS":"UNIT_STATUS", "U_CAPACITY":"CAPACITY", "CAP_OFFLINE":"CAPACITY_OFFLINE", "START_DATE": "EVENT_START", "END_DATE": "EVENT_END", 
                 "PRECISION": "DATE_PRECISION", "E_STATUS": "EVENT_STATUS", "E_CAUSE": "EVENT_CAUSE", "E_DURATION" : "EVENT_DURATION", "PADD_REG": "PAD_DISTRICT"}

In [6]:
connection_string = create_connection_string(connection_type,dataportal_db_user,dataportal_db_password,dataportal_db_ip,db_port,db_prod_name, db_driver)
engine = create_engine(connection_string, poolclass=StaticPool)

In [9]:
#Manual Run
filepath = path + filename
df = pd.read_csv(filepath, low_memory=False)[columns]
mapOfflineEventsDataframe(df, filename)

In [10]:
df = df.rename(columns = renamed_columns)

In [11]:
# Insert to SQL
df.to_sql('data_iir_offlineevents', connection_string, if_exists='append', index=False,
        dtype={ 'PUBLISH_DATE' :   sa.DateTime(),
                'EVENT_ID' :       sa.types.INTEGER(),
                'UNIT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_ID' :       sa.types.INTEGER(),
                'AREA_ID' :       sa.types.INTEGER(),
                'AREA_NAME' :     sa.types.VARCHAR(100),
                'OWNER_NAME' :     sa.types.VARCHAR(100),
                'PLANT_ID' :       sa.types.INTEGER(),
                'PLANT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_STATUS' : sa.types.VARCHAR(30),
                'UTYPE_DESC' : sa.types.VARCHAR(100),
                'CAPACITY' : sa.types.INTEGER(),
                'CAPACITY_OFFLINE' : sa.types.INTEGER(),
                'EVENT_START' :  sa.DateTime(),
                'EVENT_END' :  sa.DateTime(),
                'DATE_PRECISION' : sa.types.VARCHAR(16) ,
                'EVENT_TYPE' : sa.types.VARCHAR(30),         
                'EVENT_STATUS' : sa.types.VARCHAR(30),
                'EVENT_CAUSE' : sa.types.VARCHAR(65),
                'EVENT_DURATION' : sa.types.INTEGER(),
                'PAD_DISTRICT' : sa.types.VARCHAR(65),
                'COMMENTS' : sa.types.TEXT ,
                'ORIGIN_FILE' : sa.types.VARCHAR(65)})

##### Load for Plant File

In [7]:
filename = 'PLANT_20200801.csv'
path = 'C:\\Temp\\IIR\\'
filepath = path + filename
df = pd.read_csv(filepath, low_memory=False)

In [8]:
df.insert(0,'PUBLISH_DATE', datetime.date(2020,8, 1))
df['ORIGIN_FILE'] = filename
df = df.rename(columns={"STARTUP":"STARTUP_DATE", "SHUTDOWN":"SHUTDOWN_DATE", "UNION":"UNION_FACILITY", "COUNT":"COUNT_NUMBER"})

In [5]:
connection_string = create_connection_string(connection_type,dataportal_db_user,dataportal_db_password,dataportal_db_ip,db_port,db_prod_name, db_driver)
engine = create_engine(connection_string, poolclass=StaticPool)

In [30]:
#df.to_sql('data_iir_plant_reference', connection_string, if_exists='append', index=False)

In [28]:
# #For converting column to nullable integers use (capital Int):
# # All int columns
# df["PARENT_ID"] = df["PARENT_ID"].astype('Int64')
# df["OWNER_ID2"] = df["OWNER_ID2"].astype('Int64')
# df["PLT_MGR_ID"] = df["PLT_MGR_ID"].astype('Int64')
# df["MAINT_ID"] = df["MAINT_ID"].astype('Int64')
# df["BUYER_ID"] = df["BUYER_ID"].astype('Int64')
# df["ENGR_ID"] = df["ENGR_ID"].astype('Int64')
# df["ENVIR_ID"] = df["ENVIR_ID"].astype('Int64')
# df["SAFETY_ID"] = df["SAFETY_ID"].astype('Int64')
# df["UTILITY_ID"] = df["UTILITY_ID"].astype('Int64')
# df["IT_ID"] = df["IT_ID"].astype('Int64')
# df["HR_ID"] = df["HR_ID"].astype('Int64')
# df["NO_EMP"] = df["NO_EMP"].astype('Int64')
# df["SHIFTS_DAY"] = df["SHIFTS_DAY"].astype('Int64')
# df["MNT_STAFF"] = df["MNT_STAFF"].astype('Int64')
# df["CAPACITY"] = df["CAPACITY"].astype('Int64')
# df["ELECT_CONN"] = df["ELECT_CONN"].astype('Int64')
# df["PRIM_BUS"] = df["PRIM_BUS"].astype('Int64')
# df["ALL_BUS"] = df["ALL_BUS"].astype('Int64')

In [ ]:
#pd.set_option('display.max_rows', 182)
#df.dtypes

In [ ]:
#df[pd.isna(df["PARENT_ID"])]
#df[df["PLANT_ID"] == 2016126]
#df[df["PLANT_ID"] == 2016126]["BUYCONT_OS"]
#df.select_dtypes(include=['object'])

In [9]:
# Force convert to dates
df["PLCONT_QC"] = pd.to_datetime(df["PLCONT_QC"], format='%Y%m%d')
df["MNTCONT_QC"] = pd.to_datetime(df["MNTCONT_QC"], format='%Y%m%d')
df["BUYCONT_QC"] = pd.to_datetime(df["BUYCONT_QC"], format='%Y%m%d')
df["ENGCONT_QC"] = pd.to_datetime(df["ENGCONT_QC"], format='%Y%m%d')
df["ENVCONT_QC"] = pd.to_datetime(df["ENVCONT_QC"], format='%Y%m%d')
df["SAFCONT_QC"] = pd.to_datetime(df["SAFCONT_QC"], format='%Y%m%d')
df["UTCONT_QC"] = pd.to_datetime(df["UTCONT_QC"], format='%Y%m%d')
df["ITCONT_QC"] = pd.to_datetime(df["ITCONT_QC"], format='%Y%m%d')
df["HRCONT_QC"] = pd.to_datetime(df["HRCONT_QC"], format='%Y%m%d')
df["STARTUP_DATE"] = pd.to_datetime(df["STARTUP_DATE"], format='%Y%m%d')
df["SHUTDOWN_DATE"] = pd.to_datetime(df["SHUTDOWN_DATE"], format='%Y%m%d')
df["KICKOFF"] = pd.to_datetime(df["KICKOFF"], format='%Y%m%d')
df["QC_DATE"] = pd.to_datetime(df["QC_DATE"], format='%Y%m%d')
df["UP_DATE"] = pd.to_datetime(df["UP_DATE"], format='%Y%m%d')

In [8]:
# UNION column is defined as boolean, not sure why value 2 is in there. Force change to 1
#df.loc[df["UNION_FACILITY"] == 2, "UNION_FACILITY"] = 1

In [ ]:
#Check example
#df[df["PLANT_ID"] == 2016126]
#df[pd.isna(df["PARENT_ID"])]

In [ ]:
#df[df["PLANT_ID"] == 2016126]
#for index, row in df.iterrows():
#    currentdf = df[df["PLANT_ID"] == row.PLANT_ID]
df.to_sql('data_iir_plant_reference', connection_string, if_exists='append', index=False,
        dtype={'PUBLISH_DATE' :   sa.DateTime(),
'PLANT_ID' :       sa.types.Integer,
'PARENT_ID' :      sa.types.INTEGER(),
'PARENTNAME' :     sa.types.VARCHAR(100),
'OWNER_ID1' :      sa.types.INTEGER(),
'OWNER_NAME' :     sa.types.VARCHAR(100),
'OWNER_ID2' :      sa.types.INTEGER(),
'OWN_NAME2' :      sa.types.VARCHAR(100),
'OPER_ID' :               sa.types.INTEGER(),
'OPER_NAME' :      sa.types.VARCHAR(100),
'PLANT_NAME' :     sa.types.VARCHAR(100),
'PHYS_ADDR' :      sa.types.VARCHAR(255),
'PHYS_ADDR2' :     sa.types.VARCHAR(255),
'PHYS_CITY' :      sa.types.VARCHAR(95),
'PHYS_STATE' :     sa.types.VARCHAR(20),
'P_ST_NAME' :      sa.types.VARCHAR(95),
'PHYS_ZIP' :       sa.types.VARCHAR(20),
'PHYS_CSZ' :       sa.types.VARCHAR(95),
'P_COUNTRY' :      sa.types.VARCHAR(95),
'MAIL_ADDR' :      sa.types.VARCHAR(255),
'MAIL_ADDR2' :     sa.types.VARCHAR(255),
'MAIL_CITY' :      sa.types.VARCHAR(95),
'MAIL_STATE' :     sa.types.VARCHAR(20),
'M_ST_NAME' :      sa.types.VARCHAR(100),
'MAIL_ZIP' :       sa.types.VARCHAR(95),
'MAIL_CSZ' :       sa.types.VARCHAR(95),
'M_COUNTRY' :      sa.types.VARCHAR(95),
'COUNTY_ID' :      sa.types.VARCHAR(20),
'COUNTYNAME' :     sa.types.VARCHAR(95),
'PEC_ZONE' :       sa.types.VARCHAR(95),
'COMPLEX_RG' :     sa.types.VARCHAR(95),
'MARKET_REG' :     sa.types.VARCHAR(95),
'NERC_REGIO' :     sa.types.VARCHAR(95),
'NERC_SRGN' :      sa.types.VARCHAR(95),
'CTRLAREAID' :     sa.types.VARCHAR(95),
'CTLAREADES' :     sa.types.VARCHAR(95),
'PAD_DIST' :       sa.types.VARCHAR(95),
'PHONE' :          sa.types.VARCHAR(95),
'FAX' :            sa.types.VARCHAR(95),
'IND_CODE' :       sa.types.VARCHAR(20),
'IND_DESC' :       sa.types.VARCHAR(95),
'SIC_CODE' :       sa.types.INTEGER(),
'SIC_DESC' :       sa.types.VARCHAR(95),
'PL_STATUS' :      sa.types.VARCHAR(95),
'PLT_MGR_ID' :     sa.types.INTEGER(),
'PLANT_TLE' :      sa.types.VARCHAR(95),
'PLANT_FNME' :     sa.types.VARCHAR(95),
'PLANT_MNME' :     sa.types.VARCHAR(95),
'PLANT_LNME' :     sa.types.VARCHAR(95),
'PLANT_JRSR' :     sa.types.VARCHAR(95),
'PLCONT_PH' :      sa.types.VARCHAR(95),
'PLCONT_EX' :      sa.types.VARCHAR(95),
'PLANTEMAIL' :     sa.types.VARCHAR(95),
'PLCONT_OS' :      sa.types.Boolean,
'PLCONT_QC' :      sa.DateTime(),
'MAINT_ID' :       sa.types.INTEGER(),
'MAINT_TLE' :      sa.types.VARCHAR(95),
'MAINT_FNME' :     sa.types.VARCHAR(95),
'MAINT_MNME' :     sa.types.VARCHAR(95),
'MAINT_LNME' :     sa.types.VARCHAR(95),
'MAINT_JRSR' :     sa.types.VARCHAR(95),
'MNTCONT_PH' :     sa.types.VARCHAR(95),
'MNTCONT_EX' :     sa.types.VARCHAR(95),
'MAINTEMAIL' :     sa.types.VARCHAR(95),
'MNTCONT_OS' :     sa.types.Boolean,
'MNTCONT_QC' :     sa.DateTime(),
'BUYER_ID' :       sa.types.INTEGER(),
'BUYER_TLE' :      sa.types.VARCHAR(95),
'BUYER_FNME' :     sa.types.VARCHAR(95),
'BUYER_MNME' :     sa.types.VARCHAR(95),
'BUYER_LNME' :     sa.types.VARCHAR(95),
'BUYER_JRSR' :     sa.types.VARCHAR(95),
'BUYCONT_PH' :     sa.types.VARCHAR(95),
'BUYCONT_EX' :     sa.types.VARCHAR(95),
'BUYEREMAIL' :     sa.types.VARCHAR(95),
'BUYCONT_OS' :     sa.types.Boolean,
'BUYCONT_QC' :     sa.DateTime(),
'ENGR_ID' :               sa.types.INTEGER(),
'ENGR_TLE' :       sa.types.VARCHAR(95),
'ENGR_FNME' :      sa.types.VARCHAR(95),
'ENGR_MNME' :      sa.types.VARCHAR(95),
'ENGR_LNME' :      sa.types.VARCHAR(95),
'ENGR_JRSR' :      sa.types.VARCHAR(95),
'ENGCONT_PH' :     sa.types.VARCHAR(95),
'ENGCONT_EX' :     sa.types.VARCHAR(95),
'ENGR_EMAIL' :     sa.types.VARCHAR(95),
'ENGCONT_OS' :     sa.types.Boolean,
'ENGCONT_QC' :     sa.DateTime(),
'ENVIR_ID' :       sa.types.INTEGER(),
'ENVIR_TLE' :      sa.types.VARCHAR(95),
'ENVIR_FNME' :     sa.types.VARCHAR(95),
'ENVIR_MNME' :     sa.types.VARCHAR(95),
'ENVIR_LNME' :     sa.types.VARCHAR(95),
'ENVIR_JRSR' :     sa.types.VARCHAR(95),
'ENVCONT_PH' :     sa.types.VARCHAR(95),
'ENVCONT_EX' :     sa.types.VARCHAR(95),
'ENVIREMAIL' :     sa.types.VARCHAR(95),
'ENVCONT_OS' :     sa.types.Boolean,
'ENVCONT_QC' :     sa.DateTime(),
'SAFETY_ID' :      sa.types.INTEGER(),
'SAFE_TITLE' :     sa.types.VARCHAR(95),
'SAFE_FNME' :      sa.types.VARCHAR(95),
'SAFE_MNME' :      sa.types.VARCHAR(95),
'SAFE_LNME' :      sa.types.VARCHAR(95),
'SAFE_JRSR' :      sa.types.VARCHAR(95),
'SAFCONT_PH' :     sa.types.VARCHAR(95),
'SAFCONT_EX' :     sa.types.VARCHAR(95),
'SAFE_EMAIL' :     sa.types.VARCHAR(95),
'SAFCONT_OS' :     sa.types.Boolean,
'SAFCONT_QC' :     sa.DateTime(),
'UTILITY_ID' :     sa.types.INTEGER(),
'UTIL_TLE' :       sa.types.VARCHAR(95),
'UTIL_FNME' :      sa.types.VARCHAR(95),
'UTIL_MNME' :      sa.types.VARCHAR(95),
'UTIL_LNME' :      sa.types.VARCHAR(95),
'UTIL_JRSR' :      sa.types.VARCHAR(95),
'UTCONT_PH' :      sa.types.VARCHAR(95),
'UTCONT_EX' :      sa.types.VARCHAR(95),
'UTIL_EMAIL' :     sa.types.VARCHAR(95),
'UTCONT_OS' :      sa.types.Boolean,
'UTCONT_QC' :      sa.DateTime(),
'IT_ID' :   sa.types.INTEGER(),
'IT_TITLE' :       sa.types.VARCHAR(95),
'IT_FNME' :        sa.types.VARCHAR(95),
'IT_MNME' :        sa.types.VARCHAR(95),
'IT_LNME' :        sa.types.VARCHAR(95),
'IT_JRSR' :        sa.types.VARCHAR(95),
'ITCONT_PH' :      sa.types.VARCHAR(95),
'ITCONT_EX' :      sa.types.VARCHAR(95),
'IT_EMAIL' :       sa.types.VARCHAR(95),
'ITCONT_OS' :      sa.types.Boolean,
'ITCONT_QC' :      sa.DateTime(),
'HR_ID' :          sa.types.INTEGER(),
'HR_TITLE' :       sa.types.VARCHAR(95),
'HR_FNME' :        sa.types.VARCHAR(95),
'HR_MNME' :        sa.types.VARCHAR(95),
'HR_LNME' :        sa.types.VARCHAR(95),
'HR_JRSR' :        sa.types.VARCHAR(95),
'HRCONT_PH' :      sa.types.VARCHAR(95),
'HRCONT_EX' :      sa.types.VARCHAR(95),
'HR_EMAIL' :       sa.types.VARCHAR(95),
'HRCONT_OS' :      sa.types.Boolean,
'HRCONT_QC' :      sa.DateTime(),
'NO_EMP' :  sa.types.INTEGER(),
'SHIFTS_DAY' :     sa.types.INTEGER(),
'MNT_STAFF' :      sa.types.INTEGER(),
'MNT_CNTRCT' :     sa.types.VARCHAR(20),
'STARTUP_DATE' :   sa.DateTime(),  
'SU_PRECISN' :     sa.types.VARCHAR(16),
'SHUTDOWN_DATE' :  sa.DateTime(),  
'SD_PRECISN' :     sa.types.VARCHAR(16),
'KICKOFF' :        sa.DateTime(),
'KO_PRECISN' :     sa.types.VARCHAR(16),
'UNION_FACILITY' :        sa.types.INTEGER(),   
'CAPACITY' :       sa.types.INTEGER(),
'CAPACITY_U' :     sa.types.VARCHAR(20), 
'WASTEWATER' :     sa.types.Boolean,
'FUEL_TYPE1' :     sa.types.VARCHAR(95),
'FUEL_TYPE2' :     sa.types.VARCHAR(95),
'ELECT_GEN' :      sa.types.Boolean,
'STEAM_PROD' :     sa.types.Boolean,
'ELECT_CONN' :     sa.types.INTEGER(),
'ELECONDESC' :     sa.types.VARCHAR(95),
'ELECCONSUM' :     sa.types.VARCHAR(20),
'PRIM_BUS' :       sa.types.INTEGER(),
'ALL_BUS' :        sa.types.INTEGER(),
'NG_CONN' :        sa.types.INTEGER(),
'NGCONDESC' :      sa.types.VARCHAR(95),
'NGCONSUM' :       sa.types.VARCHAR(20),
'TR_HWY' :         sa.types.Boolean,
'TR_RAIL' :        sa.types.Boolean,
'TR_DOCK' :        sa.types.Boolean,
'TR_PIPE' :        sa.types.Boolean,
'TR_SPUR' :        sa.types.Boolean,
'TR_PORT' :        sa.types.Boolean,
'TR_TANK' :        sa.types.Boolean,
'QC_DATE' :        sa.DateTime(),
'UP_DATE' :        sa.DateTime(),
'COUNT_NUMBER' :          sa.types.INTEGER(),
'LATITUDE' :       sa.Float(),
'LONGITUDE' :      sa.Float(),
'ORIGIN_FILE' :    sa.types.VARCHAR(95)
    })

##### Load for UNIT File

In [17]:
filename = 'UNIT_20200801.csv'
path = 'C:\\Temp\\IIR\\'
filepath = path + filename
df = pd.read_csv(filepath, low_memory=False)

In [19]:
renamed_columns={"U_STATUS":"UNIT_STATUS", "STARTUP":"STARTUP_DATE", "U_CAPACITY":"CAPACITY", "COUNT" : "COUNT_NUMBER"}

In [20]:
df.insert(0,'PUBLISH_DATE', datetime.date(2020,8, 1))
df = df.rename(columns=renamed_columns)
df['ORIGIN_FILE'] = filename
df["STARTUP_DATE"] = pd.to_datetime(df["STARTUP_DATE"], format='%Y%m%d')

In [25]:
#for index, row in df.iterrows():
#    currentdf = df[df["UNIT_ID"] == row.UNIT_ID]
df.to_sql('data_iir_unit_reference', connection_string, if_exists='append', index=False,
    dtype={ 'PUBLISH_DATE' :   sa.DateTime(),
            'UNIT_ID' : sa.types.INTEGER(),
            'UNIT_NAME' : sa.types.VARCHAR(100),
            'AREA_ID' : sa.types.INTEGER(),
            'AREA_NAME' : sa.types.VARCHAR(100),
            'PLANT_ID' :sa.types.INTEGER(),
            'PLANT_NAME' : sa.types.VARCHAR(100),
            'OWNER_ID' : sa.types.INTEGER(),
            'OWNER_NAME' : sa.types.VARCHAR(100),
            'COUNTY_ID' : sa.types.VARCHAR(8),
            'UNIT_STATE' : sa.types.VARCHAR(6),
            'UNIT_STATUS' : sa.types.VARCHAR(20),  
            'STARTUP_DATE' : sa.DateTime(), 
            'UNIT_TYPE' : sa.types.VARCHAR(50),
            'CAPACITY' : sa.types.INTEGER(),
            'CAP_UOM' : sa.types.VARCHAR(12),
            'SIC_CODE' : sa.types.VARCHAR(4),
            'PAD_DIST' : sa.types.INTEGER(),
            'COUNT_NUMBER' : sa.types.INTEGER(),
            'ORIGIN_FILE' : sa.types.VARCHAR(65)})

##### Load for TAROUND file

In [26]:
#Define columns we're interested in from the file
columns=['DELV_DATE', 'OUTAGE_ID', 'UNIT_NAME', 'UNIT_ID', 'AREA_ID', 'AREA_NAME', 'OWNER_NAME', 'PLANT_ID', 'PLANT_NAME', 
         'U_STATUS', 'UTYPE_DESC', 'CHARGERATE', 'CAPACITY', 'TA_START', 'TA_END', 'PRECISION', 'OUTAGE_TYP', 
         'OUTAGE_STA', 'OUT_CAUSE', 'PERIOD', 'PAD_DIST', 'COMMENTS']

In [27]:
renamed_columns={"DELV_DATE": "PUBLISH_DATE", "OUTAGE_ID": "EVENT_ID", "U_STATUS":"UNIT_STATUS", 
                 "CHARGERATE":"CAPACITY", "CAPACITY":"CAPACITY_OFFLINE", 
                 "TA_START": "EVENT_START", "TA_END": "EVENT_END", "PRECISION": "DATE_PRECISION", "OUTAGE_TYP" : "EVENT_TYPE",
                 "OUTAGE_STA": "EVENT_STATUS", "OUT_CAUSE": "EVENT_CAUSE", "PERIOD" : "EVENT_DURATION", "PAD_DIST": "PAD_DISTRICT"}

In [28]:
filename = 'TAROUND_202008250103.csv'
path = 'C:\\Temp\\IIR\\'
filepath = path + filename
df = pd.read_csv(filepath, low_memory=False)[columns]

In [29]:
df = df.rename(columns=renamed_columns)

In [30]:
df["PUBLISH_DATE"] = pd.to_datetime(df["PUBLISH_DATE"], format='%Y%m%d')
df["EVENT_START"] = pd.to_datetime(df["EVENT_START"], format='%Y%m%d')
df["EVENT_END"] = pd.to_datetime(df["EVENT_END"], format='%Y%m%d')
df['ORIGIN_FILE'] = filename

In [ ]:
# Insert to SQL
df.to_sql('data_iir_offlineevents', connection_string, if_exists='append', index=False,
        dtype={ 'PUBLISH_DATE' :   sa.DateTime(),
                'EVENT_ID' :       sa.types.INTEGER(),
                'UNIT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_ID' :       sa.types.INTEGER(),
                'AREA_ID' :       sa.types.INTEGER(),
                'AREA_NAME' :     sa.types.VARCHAR(100),
                'OWNER_NAME' :     sa.types.VARCHAR(100),
                'PLANT_ID' :       sa.types.INTEGER(),
                'PLANT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_STATUS' : sa.types.VARCHAR(30),
                'UTYPE_DESC' : sa.types.VARCHAR(100),
                'CAPACITY' : sa.types.INTEGER(),
                'CAPACITY_OFFLINE' : sa.types.INTEGER(),
                'EVENT_START' :  sa.DateTime(),
                'EVENT_END' :  sa.DateTime(),
                'DATE_PRECISION' : sa.types.VARCHAR(16) ,
                'EVENT_TYPE' : sa.types.VARCHAR(30),         
                'EVENT_STATUS' : sa.types.VARCHAR(30),
                'EVENT_CAUSE' : sa.types.VARCHAR(65),
                'EVENT_DURATION' : sa.types.INTEGER(),
                'PAD_DISTRICT' : sa.types.VARCHAR(65),
                'COMMENTS' : sa.types.VARCHAR(2000) ,
                'ORIGIN_FILE' : sa.types.VARCHAR(65)})

In [3]:
def insertTaroundFile(filename, path):
    filepath = path + filename
    #Define columns we're interested in from the file
    columns=['DELV_DATE', 'OUTAGE_ID', 'UNIT_NAME', 'UNIT_ID', 'AREA_ID', 'AREA_NAME', 'OWNER_NAME', 'PLANT_ID', 'PLANT_NAME', 
             'U_STATUS', 'UTYPE_DESC', 'CHARGERATE', 'CAPACITY', 'TA_START', 'TA_END', 'PRECISION', 'OUTAGE_TYP', 
             'OUTAGE_STA', 'OUT_CAUSE', 'PERIOD', 'PAD_DIST', 'COMMENTS']
    renamed_columns={"DELV_DATE": "PUBLISH_DATE", "OUTAGE_ID": "EVENT_ID", "U_STATUS":"UNIT_STATUS", 
                 "CHARGERATE":"CAPACITY", "CAPACITY":"CAPACITY_OFFLINE", 
                 "TA_START": "EVENT_START", "TA_END": "EVENT_END", "PRECISION": "DATE_PRECISION", "OUTAGE_TYP" : "EVENT_TYPE",
                 "OUTAGE_STA": "EVENT_STATUS", "OUT_CAUSE": "EVENT_CAUSE", "PERIOD" : "EVENT_DURATION", "PAD_DIST": "PAD_DISTRICT"}
    df = pd.read_csv(filepath, low_memory=False)[columns]
    df = df.rename(columns=renamed_columns)
    df["PUBLISH_DATE"] = pd.to_datetime(df["PUBLISH_DATE"], format='%Y%m%d')
    df["EVENT_START"] = pd.to_datetime(df["EVENT_START"], format='%Y%m%d')
    df["EVENT_END"] = pd.to_datetime(df["EVENT_END"], format='%Y%m%d')
    df['ORIGIN_FILE'] = filename
    df.to_sql('data_iir_offlineevents', connection_string, if_exists='append', index=False,
        dtype={ 'PUBLISH_DATE' :   sa.DateTime(),
                'EVENT_ID' :       sa.types.INTEGER(),
                'UNIT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_ID' :       sa.types.INTEGER(),
                'AREA_ID' :       sa.types.INTEGER(),
                'AREA_NAME' :     sa.types.VARCHAR(100),
                'OWNER_NAME' :     sa.types.VARCHAR(100),
                'PLANT_ID' :       sa.types.INTEGER(),
                'PLANT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_STATUS' : sa.types.VARCHAR(30),
                'UTYPE_DESC' : sa.types.VARCHAR(100),
                'CAPACITY' : sa.types.INTEGER(),
                'CAPACITY_OFFLINE' : sa.types.INTEGER(),
                'EVENT_START' :  sa.DateTime(),
                'EVENT_END' :  sa.DateTime(),
                'DATE_PRECISION' : sa.types.VARCHAR(16) ,
                'EVENT_TYPE' : sa.types.VARCHAR(30),         
                'EVENT_STATUS' : sa.types.VARCHAR(30),
                'EVENT_CAUSE' : sa.types.VARCHAR(65),
                'EVENT_DURATION' : sa.types.INTEGER(),
                'PAD_DISTRICT' : sa.types.VARCHAR(65),
                'COMMENTS' : sa.types.VARCHAR(2000) ,
                'ORIGIN_FILE' : sa.types.VARCHAR(65)})
    print("File %s uploaded successfully to the database" % filename)

#### Importing multiple files from local path

In [4]:
import os

In [16]:
path = 'C:\\Temp\\IIR\\TAROUND'
extension='CSV'
filename_pattern='TAROUND'
file_names = [fn.upper() for fn in os.listdir(path)
              if filename_pattern in fn.upper() and fn.upper().endswith(extension)]

In [54]:
sql_query = "select distinct(ORIGIN_FILE) from data_iir_offlineevents where origin_file like 'TAROUND%'"
current_db_files = [file.upper() for file in pd.read_sql(sql_query, connection_string)["ORIGIN_FILE"]]

In [ ]:
current_files

In [ ]:
for name in file_names:
    #if name not in current_db_files:
        insertTaroundFile(name, path)
        print('Inserted' + name)

#### Importing multiple files from ftp server

In [5]:
from ftplib import FTP
global ftp

ftp_site='ftp.industrialinfo.com'
user='andurand'
pw='361984'
path = 'C:\\Temp\\IIR\\TAROUND\\'
ftp_path='/data/'
extension='CSV'
filename_pattern='TAROUND'

ftp = FTP(ftp_site, user=user, passwd=pw)
ftp.cwd(ftp_path)

'250 Directory successfully changed.'

In [6]:
# Retrieve list of current inserted files from the database
sql_query = "select distinct(ORIGIN_FILE) from data_iir_offlineevents where origin_file like 'TAROUND%'"
connection_string = create_connection_string(connection_type,dataportal_db_user,dataportal_db_password,dataportal_db_ip,db_port,db_prod_name, db_driver)
current_files = [file.upper() for file in pd.read_sql(sql_query, connection_string)["ORIGIN_FILE"]]

In [18]:
[fn.upper() for fn in ftp.nlst() 
              if filename_pattern in fn.upper() and fn.upper().endswith(extension) and fn.upper() not in current_files]

['TAROUND_202008100101.CSV']

In [24]:
# # Get missing files
# file_names = [fn.upper() for fn in ftp.nlst()
#               if filename_pattern in fn.upper() and fn.upper().endswith(extension) and fn.upper() not in current_files]

In [10]:
# Download missing files and save to db
ftp = FTP(ftp_site, user=user, passwd=pw)
ftp.cwd(ftp_path)
file_names = [fn.upper() for fn in ftp.nlst() 
              if filename_pattern in fn.upper() and fn.upper().endswith(extension) and fn.upper() not in current_files]
for file_name in file_names:
    local_filename = os.path.join(path, file_name)
    if not os.path.isfile(local_filename):
        with open(local_filename, 'wb') as local_file:
            ftp.retrbinary('RETR %s' % file_name, local_file.write)
            print("Downloaded: %s" % file_name);
    else:
        print("File already exists on drive");
    
    insertTaroundFile(file_name, path)

File already exists on drive
File TAROUND_202007130102.CSV uploaded successfully to the database
File already exists on drive
File TAROUND_202007260101.CSV uploaded successfully to the database
File already exists on drive
File TAROUND_202008100101.CSV uploaded successfully to the database
File already exists on drive
File TAROUND_202008160100.CSV uploaded successfully to the database
File already exists on drive
File TAROUND_202008170101.CSV uploaded successfully to the database
File already exists on drive
File TAROUND_202008220101.CSV uploaded successfully to the database
File already exists on drive
File TAROUND_202008240102.CSV uploaded successfully to the database
Downloaded: TAROUND_202008270103.CSV
File TAROUND_202008270103.CSV uploaded successfully to the database
Downloaded: TAROUND_202008280102.CSV
File TAROUND_202008280102.CSV uploaded successfully to the database


In [35]:
path = 'C:\\Temp\\IIR\\'
filename_pattern='TAROUND'

In [21]:
# #Delete all local files matching pattern
# path = 'C:\\Temp\\IIR\\TAROUND'
# filename_pattern='TAROUND'
# file_names = [fn.upper() for fn in os.listdir(path)
#               if filename_pattern in fn.upper() and fn.upper().endswith(extension)]
# for name in file_names:
#     filepath = os.path.join(path, name)
#     try:
#         os.remove(filepath)
#     except:
#         print("Error while deleting file : ", filepath)

In [57]:
file_names

[]

In [25]:
local_filename = os.path.join(path, name)

In [39]:
with open(local_filename, 'wb') as f:
        ftp.retrbinary('RETR %s' % name, f.write)
        print("Downloaded: %s" % name);    

Downloaded: TAROUND_202008260102.CSV
